In [1]:
!nvidia-smi
from tensorflow.keras import layers
import tensorflow as tf

Sun Mar  3 13:52:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.58                 Driver Version: 537.58       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   39C    P0              16W /  80W |      0MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

In [3]:
#!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

In [4]:
#unzip_data('nlp_getting_started.zip')

In [5]:
import pandas as pd
train_df = pd.read_csv("train.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df_shuffled = train_df.sample(frac=1, random_state = 42)

In [7]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
test_df = pd.read_csv("test.csv")
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [10]:
len(train_df),len(test_df)

(7613, 3263)

In [11]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _,text,target = row
    print(f"Target : {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text : \n{text}\n")
    print("----\n")
    


Target : 0 (not real disaster)
Text : 
He made such a good point. White person comings mass murder labelled as criminal minority does the same thing... http://t.co/37qPsSnaCv

----

Target : 1 (real disaster)
Text : 
Skinny Jeans are Hazardous for Your Health! #socialnews http://t.co/92Pk0HujD8

----

Target : 0 (not real disaster)
Text : 
https://t.co/eCMUjkKqX1 @ArianaGrande @ScreamQueens 
Katherine's Death

----

Target : 0 (not real disaster)
Text : 
@TinyJecht Are you another Stand-user? If you are I will have to detonate you with my Killer Queen.

----

Target : 1 (real disaster)
Text : 
70 Years After Atomic Bombs Japan Still Struggles With War Past http://t.co/5wfXbAQMBK The anniversary of the devastation wrought by theÛ_

----



In [12]:
#007 Splitting data into training and validation sets.mp4
from sklearn.model_selection import train_test_split
train_sentences , val_sentences ,train_labels,val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size = 0.1,
                                                                            random_state = 42)
                                                                            
                                                                            

In [13]:
len(train_sentences),len(train_labels),len(val_sentences),len(val_labels)

(6851, 6851, 762, 762)

In [14]:
#008 Converting text data to numbers using tokenisation and embeddings (overview).mp4
#009 Setting up a TensorFlow TextVectorization layer to convert text to numbers.mp4
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
#OOV  = "Out of Vocabulary"
text_vectorizer = TextVectorization(max_tokens=10000,
                                    standardize = "lower_and_strip_punctuation",
                                    split = "whitespace",
                                    ngrams = None,
                                    output_mode = "int",
                                    output_sequence_length=None,
                                    pad_to_max_tokens=True)
                                    
                                    
                                    

In [15]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length,
                                   )

In [17]:
#010 Mapping the TextVectorization layer to text data and turning it into numbers
# Fit the text vectorizer to training text
text_vectorizer.adapt(train_sentences)

In [18]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\\n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 @God how come bugs haven't all drowned out of existence every time it rains?\n
Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 200,   62,  220,    1, 1134,   44,  605,   36,    6, 3873,  197,
          92,   15, 2330,    0]], dtype=int64)>

In [19]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
#[UNK] meand unknown token

In [20]:
len(words_in_vocab)

10000

In [21]:
#011 Creating an Embedding layer to turn tokenised text into embedding vectors
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length = max_length
                             )

In [22]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\\n\nEmbedded version:")
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Top link: Reddit's new content policy goes into effect many horrible subreddits banned or quarantined http://t.co/zCp5cszSLl\n
Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 3.58633660e-02,  1.22083798e-02,  1.45695247e-02, ...,
         -3.72178331e-02, -4.25874591e-02,  2.45838799e-02],
        [-4.20822725e-02,  4.59202267e-02, -4.91449125e-02, ...,
          8.11231136e-03,  7.63509423e-03, -1.33704394e-04],
        [ 3.74884941e-02,  8.90158117e-04, -2.20639631e-03, ...,
         -4.13975492e-02,  3.76926325e-02, -2.08885670e-02],
        ...,
        [ 6.98678195e-05,  4.15689684e-02,  6.21036440e-03, ...,
         -3.60144377e-02,  1.17313266e-02, -8.41363519e-03],
        [-1.55090317e-02, -2.41957791e-02,  2.30121724e-02, ...,
          3.15246321e-02, -1.47824995e-02,  4.83939983e-02],
        [ 1.67775489e-02,  4.22526710e-02,  2.41547339e-02, ...,
          3.79778631e-02,  3.20063941e-02,  3.34445722e-02]]],
      dtype=float32)>

In [23]:
#check out a single token embedding
sample_embed[0][0],sample_embed[0][0].shape,random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.03586337,  0.01220838,  0.01456952, -0.00856993, -0.00633182,
         0.01241682,  0.00204645, -0.00108371, -0.00163586, -0.04919105,
         0.01323109,  0.03442483, -0.02257118,  0.00859853,  0.03848143,
         0.01416883,  0.01878599, -0.00056169, -0.03299709,  0.01071971,
        -0.01266847, -0.03614638, -0.01174623,  0.04208029,  0.04506264,
         0.00797756,  0.04917815,  0.00082575,  0.01765509,  0.00452513,
         0.0326038 ,  0.0455527 , -0.02370201, -0.03624595,  0.01728417,
        -0.04453979, -0.04746628,  0.00793644,  0.04606395,  0.02751962,
         0.00799175,  0.00526165, -0.00566538, -0.00771046, -0.00938361,
        -0.02554187, -0.01237558, -0.01261739,  0.02826818, -0.03763397,
        -0.00977149, -0.02917041,  0.00397171, -0.02918229,  0.04617893,
         0.02881292,  0.00288584, -0.01712673, -0.03522921, -0.04581093,
        -0.00368816,  0.00879891, -0.02871103,  0.02721765,  0.01789859,
  

In [24]:
#012 Discussing the various modelling experiments we're going to run
#013 Model 0 Building a baseline model to try and improve upon
#Multinomial naive bais using TF_IDF formula
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf",MultinomialNB()),

])
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
#Evalute our baseline model
baseline_score = model_0.score(val_sentences,val_labels)
print(f"Our baseline model achives an accuracy of : {baseline_score*100:.2f}%")

Our baseline model achives an accuracy of : 79.27%


In [26]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int64)

In [27]:
#014 Creating a function to track and evaluate our model's results
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_results(y_true,y_pred):
    model_accuracy = accuracy_score(y_true,y_pred)*100
    model_precision,model_recall,model_f1,_=precision_recall_fscore_support(y_true, y_pred,average="weighted")
    model_results = {"accuracy":model_accuracy,
                     "precision":model_precision,
                     "recall":model_recall,
                     "f1":model_f1}
    return model_results

In [28]:
baseline_results = calculate_results(y_true=val_labels,y_pred=baseline_preds)
baseline_results
#higher precision leads to less false positives
#higher recall leads to less false negetives

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [29]:
#015 Model 1 Building, fitting and evaluating our first deep model on text data
from helper_functions import create_tensorboard_callback
from tensorflow.keras import layers
SAVE_DIR = "model_logs"
#build model with Function API
inputs = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs) # turn input text into numbers
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x) #layers.Reshape((1920,))(x) #.
outputs = layers.Dense(1,activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name = "model_1_dense")
model_1.summary()



Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [30]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20240303-135307
Epoch 1/5
215/215 [==============================] - 4s 5ms/step - loss: 0.6116 - accuracy: 0.6875 - val_loss: 0.5334 - val_accuracy: 0.7651
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.4417 - accuracy: 0.8173 - val_loss: 0.4690 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3471 - accuracy: 0.8619 - val_loss: 0.4575 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2848 - accuracy: 0.8889 - val_loss: 0.4634 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2381 - accuracy: 0.9118 - val_loss: 0.4803 - val_accuracy: 0.7848


In [31]:
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4803 - accuracy: 0.7848


[0.4802533984184265, 0.7847769260406494]

In [32]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[0]

24/24 [==============================] - 0s 1ms/step


array([0.45384398], dtype=float32)

In [33]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [34]:
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [35]:
#calculate model 1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.4776902887139,
 'precision': 0.789165199286798,
 'recall': 0.7847769028871391,
 'f1': 0.7818959205825942}

In [36]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [37]:
#016 Visualising our model's learned word embeddings with TensorFlow's projector tool
words_in_vocab = text_vectorizer.get_vocabulary()

In [38]:
len(words_in_vocab),words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [39]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [40]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape
#https://projector.tensorflow.org/
#https://www.tensorflow.org/text/guide/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk

(10000, 128)

In [41]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [55]:
#017 High-level overview of Recurrent Neural Networks (RNNs) + where to learn more
#018 Model 2 Building, fitting and evaluating our first TensorFlow RNN model (LSTM)
#MOdel 2 LSTM
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.LSTM(units=64,return_sequences=True)(x)
# x = layers.LSTM(units=64,return_sequences=True)(x)
x = layers.LSTM(64)(x)
# x = layers.Dense(64,activation="relu")(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs,name="model_2_LSTM")


In [56]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm_18 (LSTM)              (None, 64)                49408     
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [58]:
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                       "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20240303-143732
Epoch 1/5
215/215 [==============================] - 6s 9ms/step - loss: 0.2275 - accuracy: 0.9150 - val_loss: 0.6420 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1609 - accuracy: 0.9404 - val_loss: 0.5771 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1309 - accuracy: 0.9517 - val_loss: 0.6249 - val_accuracy: 0.7861
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1019 - accuracy: 0.9618 - val_loss: 0.7914 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0860 - accuracy: 0.9680 - val_loss: 1.0234 - val_accuracy: 0.7769


In [62]:
#make prediction with LSTM Model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 4ms/step


array([[9.8144542e-04],
       [7.7120656e-01],
       [9.9971849e-01],
       [1.6836988e-02],
       [3.3921946e-04],
       [9.9533468e-01],
       [5.0105220e-01],
       [9.9982268e-01],
       [9.9968708e-01],
       [4.8435986e-01]], dtype=float32)

In [63]:
model_2_pred=tf.squeeze(tf.round(model_2_pred_probs))

In [65]:
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_pred)

In [66]:
model_2_results

{'accuracy': 77.69028871391076,
 'precision': 0.7809693289921038,
 'recall': 0.7769028871391076,
 'f1': 0.7739165030429329}

In [67]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [79]:
#019 Model 3 Building, fitting and evaluating a GRU-cell powered RNN
#Gated Recurrent Unit
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
# x= layers.LSTM(64,return_sequences=True)(x)
# x = layers.GRU(64)(x)
# x= layers.GlobalAveragePooling1D()(x)
x = layers.Dense(64,activation='relu')(x)
outputs = layers.Dense(1,activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs,outputs,name = "model_3_GRU")

In [80]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       multiple                  1280000   
                                                                 
 gru_14 (GRU)                (None, 64)                37248     
                                                                 
 dense_24 (Dense)            (None, 64)                4160      
                                                                 
 dense_25 (Dense)            (None, 1)                 65        
                                                       

In [81]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                       "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20240303-161533
Epoch 1/5
215/215 [==============================] - 3s 9ms/step - loss: 0.1440 - accuracy: 0.9451 - val_loss: 0.8817 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0839 - accuracy: 0.9692 - val_loss: 0.8982 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0675 - accuracy: 0.9731 - val_loss: 1.0475 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0583 - accuracy: 0.9778 - val_loss: 1.0759 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0503 - accuracy: 0.9785 - val_loss: 1.3657 - val_accuracy: 0.7703


In [82]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred=tf.squeeze(tf.round(model_3_pred_probs))
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_pred)
model_3_results

24/24 [==============================] - 0s 4ms/step


{'accuracy': 77.03412073490814,
 'precision': 0.7726339681635592,
 'recall': 0.7703412073490814,
 'f1': 0.7679241301282296}

In [84]:
model_2_results

{'accuracy': 77.69028871391076,
 'precision': 0.7809693289921038,
 'recall': 0.7769028871391076,
 'f1': 0.7739165030429329}

In [ ]:
#020 Model 4 Building, fitting and evaluating a bidirectional RNN model